# GitHub Actions CI Setup: CUDA with CPU Fallback

This tutorial documents how to set up GitHub Actions CI for PyTorch projects that use CUDA, with automatic fallback to CPU when CUDA is not available.

## Table of Contents
1. [Setting Up GitHub Actions Workflow](#setting-up-github-actions-workflow)
2. [Implementing Device Detection in Your Code](#implementing-device-detection-in-your-code)
3. [Testing Locally](#testing-locally)
4. [Troubleshooting](#troubleshooting)

---



GitHub Actions' standard hosted runners (`ubuntu-latest`, `windows-latest`, etc.) do **not** provide GPU access or CUDA support. The solution is to:

1. **Use standard CPU-only runners** (`ubuntu-latest`) for CI
2. **Implement device detection** in your code that automatically falls back to CPU when CUDA is unavailable
3. **Test on CPU** in CI, knowing your code will work on GPU when available locally

This approach ensures:
- ✅ Tests run successfully on GitHub Actions (CPU)
- ✅ Code works on local machines with GPU (CUDA)
- ✅ Code gracefully handles both environments
- ✅ No additional costs or infrastructure needed

---

## Setting Up GitHub Actions Workflow

### Step 1: Create the Workflow File

Create a file at `.github/workflows/ci.yml` in your repository root:

```yaml
name: Run Tests

on:
  push:
    branches:
      - master
      - develop
  pull_request:
    branches:
      - master

jobs:
  test:
    runs-on: ubuntu-latest  # CPU-only runner; scripts will detect device and fall back to CPU
    steps:
      - name: Checkout repository
        uses: actions/checkout@v3

      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: "3.11"

      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install pytest numpy torch  # PyTorch CPU version; scripts handle device detection

      - name: Run tests
        run: pytest --ignore=projects
```

---

## Implementing Device Detection in Your Code

To make your code work in both GPU and CPU environments, implement device detection in your scripts.

### Basic Pattern

```python
import torch

# Detect device and fall back to CPU if CUDA is not available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Use the device for your tensors/models
model = model.to(device)
data = data.to(device)
```

### Example: Training Script

```python
import torch
import torch.nn as nn

# Device detection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}")

# Model setup
model = nn.Linear(10, 1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# Training loop
for epoch in range(10):
    # Generate dummy data on the correct device
    inputs = torch.randn(32, 10).to(device)
    targets = torch.randn(32, 1).to(device)
    
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
```


### Best Practices

1. **Always check CUDA availability**: Use `torch.cuda.is_available()` before using CUDA
2. **Move tensors to device explicitly**: Use `.to(device)` for all tensors and models
3. **Print device info**: Helpful for debugging and logging
4. **Handle device mismatches**: Ensure all tensors in operations are on the same device

---

## Testing Locally

Before pushing to GitHub, test your workflow locally using one of two methods:

### Method 1: Manual Testing (Fast & Simple)

Run the workflow commands directly in your environment:

```bash
# Navigate to your project directory
cd /home/$USERNAME/workspace/PyTorchTutorial

# Activate your conda environment (if using conda)
conda activate PyTorchTutorial

# Run the workflow steps manually
python -m pip install --upgrade pip
pip install pytest numpy torch
pytest --ignore=projects
```

### Method 2: Using `act` (Full Simulation)

`act` is a tool that runs GitHub Actions workflows locally using Docker, providing a more accurate simulation.

#### Installation

Recommended way to install **GitHub CLI (gh)** on Ubuntu 

1. **Install prerequisites**

```bash
sudo apt update
sudo apt install -y curl gpg
```

2. **Add GitHub CLI GPG key**

```bash
curl -fsSL https://cli.github.com/packages/githubcli-archive-keyring.gpg \
  | sudo dd of=/usr/share/keyrings/githubcli-archive-keyring.gpg
```

3. **Add the official apt repository**

```bash
echo "deb [arch=$(dpkg --print-architecture) \
signed-by=/usr/share/keyrings/githubcli-archive-keyring.gpg] \
https://cli.github.com/packages stable main" \
  | sudo tee /etc/apt/sources.list.d/github-cli.list > /dev/null
```

4. **Install gh**

```bash
sudo apt update
sudo apt install gh
```

5. **Verify**

```bash
gh --version
```

6. **GitHub CLI Extension**


```bash
gh extension install nektos/gh-act
```

Then use it as: `gh act` instead of just `act`


**Verify Installation:**

After installation, verify it works:

```bash
act --version
```


#### Configuration

Create a config file to avoid interactive prompts:

```bash
mkdir -p ~/.config/act
echo "-P ubuntu-latest=catthehacker/ubuntu:act-latest" > ~/.config/act/actrc
```


**Note:** `catthehacker/ubuntu:act-latest` is a pre-built Docker image maintained by the act community. It simulates GitHub Actions' Ubuntu runner environment. You don't need to change this name - it's the standard image used by `act` to replicate GitHub Actions behavior locally.



#### Running Tests

```bash
# Navigate to your project directory
cd /home/$USERNAME/workspace/PyTorchTutorial

# List what would run (dry-run)
act push --list

# Run the workflow
act push -W .github/workflows/ci.yml

# Run specific workflow file
act push -W .github/workflows/ci.yml -P ubuntu-latest=catthehacker/ubuntu:act-latest
```

**Pros:**
- Simulates exact GitHub Actions environment
- Uses Docker containers like GitHub Actions
- Catches environment-specific issues

**Cons:**
- Slower (downloads Docker images)
- Requires Docker to be installed and running
- First run takes longer

#### Common `act` Commands

```bash
# List available workflows
act -l

# Run specific job
act -j test

# Run with specific event
act push
act pull_request

# Run specific workflow file
act -W .github/workflows/ci.yml

# Show what would run without executing
act --list
```

---

